# Tutorial

The following pages show an example analysis with emgfit broken down into the essential steps. Many of the more advanced features of emgfit are left out or only briefly mentioned in passing, so feel free to explore the documentation further!  

This tutorial was created in the Jupyter Notebook `emgfit_tutorial.ipynb` which can be found in the `emgfit/emgfit/examples/tutorial/` directory of the emgfit distribution. Feel free to copy the tutorial folder to a different directory (outside of the `emgfit/` directory!) and follow along with the tutorial by actually running the code. You can also use this notebook as a template for your own analyses (consider removing some of the explanations). It is recommended to use a separate notebook for each spectrum to be fitted. This enables you to go back to the notebook at any time and check on all the details of how the fits were performed. 

emgfit is optimized to be run within Jupyter Notebooks. There is dozens of decent introductions to using Jupyter Notebooks, a nice overview can e.g. be found at https://realpython.com/jupyter-notebook-introduction/. Naturally, the first step of an analysis with emgfit is starting up your notebook server by running ``jupyter notebook`` in your command-line interface. This should automatically make the Jupyter interface pop up in a browser window. From there you can navigate to different directories and create a new notebook (`new` panel on the top right) or open an existing notebook (.ipynb files). If you installed emgfit into a virtual environment be sure to activate it before starting the notebook server and check in the notebook that the correct kernel is selected.

## Import the package
Assuming you have setup emgfit following the [installation instructions](installation.rst), the first step after launching your Jupyter Notebook will be importing the emgfit package:

In [ ]:
### Import fit package 
import emgfit as emg
print("emgfit version:",emg.__version__)

## How to access the documentation 
Before we actually start processing a spectrum it is important to know how to get access to emgfit's documentation. There is multiple options for this:

1. The **html documentation** can be viewed in any browser. It contains usage examples, detailed explanations of the crucial components and API docs with the different modules and all their methods. The search option and cross references enable quick and easy browsing for help.
2. Once you have imported emgfit you can **access the docs directly from the Jupyter Notebook**:
    * print all available methods of e.g. the spectrum class by running `dir(emg.spectrum)` 
    * print documentation of a method using `help()`, e.g. the docs of the `add_peak` method are printed by running `help(emg.spectrum.add_peak)` in a code cell
    * keyboard shortcuts can be even more convenient:
        * Use `TAB` to get suggestions for auto-completion of method and variable names
        * Place the cursor inside the brackets of a function/method and press `SHIFT` + `TAB` to have a window with the     function/method documention pop up
        * Pressing the `H` key inside a Jupyter Notebook shows you all available keyboard shortcuts)

## Import data 
The following code imports the mass data and creates an emgfit spectrum object called `spec`. The input file must be a TXT or CSV-file with the bin centers and counts per bin as the respective columns (this is in line e.g. with the format of the `hist` export mode in the mass acquisition software `MAc`). From here on the analysis of the spectrum proceeds by calling the various methods on our spectrum object `spec`.

In [ ]:
### Import mass data, plot full spectrum and indicate chosen fit range
filename = "2019-09-13_004-_006 SUMMED High stats 62Ga"
skiprows = 38 # number of header rows to skip upon data import
m_start = 61.9243 # low-mass cut off
m_stop = 61.962 # high-mass cut off

spec = emg.spectrum(filename+'.txt',m_start,m_stop,skiprows=skiprows)

## Add peaks to the spectrum 
This can be done with the automatic peak detection spectrum method) and/or by manually adding peaks ([add_peak()](modules.rst#emgfit.spectrum.spectrum.add_peak) spectrum method). The plots shown below are (optional) outputs of the  [detect_peaks()](modules.rst#emgfit.spectrum.spectrum.detect_peaks) method and depicts the different stages of the automatic peak detection.

All information about the peaks associated with the spectrum are compiled in the peak properties table. The table's left-most column shows the respective peak indeces. In all fits, the peaks' `x_pos` will be used as the initial values for the peak position parameters `mu` (to be exact: `mu` marks the centroid of the underlying Gaussians). 

In [ ]:
### Detect peaks and add them to spectrum object 'spec'
spec.detect_peaks() # automatic peak detection
#spec.add_peak(61.925,species='?') # manually add a peak at x_pos = 61.925u
#spec.remove_peak(peak_index=0) # manually remove the peak with index 0 

## Assign species to the peaks (optional)
Although this step is optional, it is highly recommended that it is not skipped. By assigning species labels to your peaks you do not only gain more overview over your spectrum, but also allow for literature values to be automatically fetched from the AME database and entered into the peak properties table. Once a `species` label has been assigned, you can refer to this peak not only via its index but also via the label.

The [assign_species()](modules.rst#emgfit.spectrum.spectrum.assign_species) method allows to assign species identifications either to a single selected peak or to all peaks at once. Here the second option was used by passing a list of species labels to [assign_species()](modules.rst#emgfit.spectrum.spectrum.assign_species). The list must have the same length as the number of peaks associated with the spectrum object. If there are peaks whose labels should not be changed (e.g. unidentified peaks), simply insert ``None`` as a placeholder at the corresponding spots (as done for peaks 2 and 7 below). The syntax for species labels follows the [:-notation](concepts.rst#notation-of-chemical-substances). **It is important not to forget to subtract the number of electrons corresponding to the ion's charge state!** Otherwise the analysis would mistakenly proceed with the atomic instead of the ionic mass. Note that currently only singly charged species are supported by emgfit. Tentative peak identifications can be indicated by adding a ``'?'`` to the end of the species string. In this case the literature values are not fetched. The user can also define custom literature values (e.g. to handle isomers or if there are recent measurements that have not entered the AME yet). For more details see the documentation of [assign_species()](modules.rst#emgfit.spectrum.spectrum.assign_species).   

This is also a good point in time to add any comments to the peaks using the [add_peak_comment()](modules.rst#emgfit.spectrum.spectrum.add_peak_comment) method. These comments can be particularly helpful for post-processing in Excel since they are also written into the output file with the fit results (as is the entire peak properties table). More general comments that concern the entire spectrum can instead be added with the [add_spectrum_comment()](modules.rst#emgfit.spectrum.spectrum.add_spectrum_comment) method. 

In [ ]:
### Assign species and add peak comments
spec.assign_species(['Ni62:-1e','Cu62:-1e?',None,'Ga62:-1e','Ti46:O16:-1e','Sc46:O16:-1e','Ca43:F19:-1e',None])
spec.add_peak_comment('Non-isobaric',peak_index=2)
spec.show_peak_properties() # check the changes by printing the peak properties table

## Activate hiding of mass values for blind analysis (optional)

By adding peak indeces to the spectrum's `blinded_peaks` list, the obtained masses and positions of selected peaks-of-interest can be hidden from the user. This blindfolding can avoid user bias and is automatically lifted once the results are exported.  

In [ ]:
### Optionally turn on blinding of specific peaks of interest to enable blind analysis 
spec.set_blinded_peaks([0,3]) # activate blinding for peaks 0 & 3
#spec.set_blinded_peaks([],overwrite=True) # run this to deactivate blinding for all peaks

## Select the optimal fit model and perform the peak-shape calibration 
Next we need to find both a fit model and a set of model parameters that capture the shape of our peaks as well as possible. In emgfit both of this is achieved with the [determine_peak_shape()](modules.rst#emgfit.spectrum.spectrum.determine_peak_shape) method. Once the peak-shape calibration has been performed all subsequent fits will be performed with this fixed peak-shape, by only varying the peak centroids, amplitudes and optionally the uniform-baseline parameter `bkg_c`.

By default [determine_peak_shape()](modules.rst#emgfit.spectrum.spectrum.determine_peak_shape) performs an **automatic model selection** in which the shape-calibrant peak is first fitted with a pure Gaussian and then with Hyper-EMG functions with an increasing number of expontential tails on the left and right. The algorithm selects the fit model which yields the smallest $\chi^2_\text{red}$ without having any of the tail weight parameters $\eta$ compatible with zero within their uncertainty. Alternatively, the auto-model selection can be turned off with the argument `vary_tail_order=False` and the fit model can be selected manually with the `fit_model` argument.

Once the best fit model has been selected the method proceeds with the determination of the peak-shape parameters and shows a detailed report with the fit results. 

Some recommendations:

- It is recommended to do the peak-shape calibration with a chi-squared fit (default) since this yields more robust     results and more trusworthy parameter uncertainty estimates. Check the method docs for info on performing the shape calibration with binned maximum likelihood estimation.
- Ideally the peak-shape calibration is performed on a well-separated peak with high statistics. In this example, the `Ca43:F19:-1e` peak was selected as peak-shape calibrant. Since the default fit range includes a smaller peak on the right, the range was manually reduced to 0.45u with the `x_fit_range` argument. If unavoidable, the peak-shape determination can also be performed on partially overlapping peaks since emgfit ensures identical shape parameters for all peaks being fitted. 

In [ ]:
## Peak-shape calibration with default settings, including automatic model selection:
#spec.determine_peak_shape(species_shape_calib='Ca43:F19:-1e') 

## Peak-shape calibration with user-defined fit range:
spec.determine_peak_shape(species_shape_calib='Ca43:F19:-1e',x_fit_range=0.0045) 

## Peak-shape calibration with user-defined fit model:
#spec.determine_peak_shape(species_shape_calib='Ca43:F19:-1e',fit_model='emg12',vary_tail_order=False)

## Determine A_stat_emg for subsequent stat. error estimations (optional)

By default, the statistical uncertainties of Hyper-EMG fits are estimated using the equation:

$\sigma_{stat} = A_{stat,emg} \cdot \frac{\mathrm{FWHM}}{\sqrt{N_{counts}}}$

where $\mathrm{FWHM}$ and $N_{counts}$ refer to the full width at half maximum and the number of counts in the respective peak. **This step can be skipped when the statistical uncertainties are estimated using the get_errors_from_resampling() method (see "Perform parametric bootstrap to get refined statistical uncertainties" section below).**

By default a of value $A_{stat,emg} = 0.52$ will be used for Hyper-EMG models (for Gaussians $A_{stat,G}=0.425$).

However, $A_{stat,emg}$ depends on the peak-shape and can deviate from the default value. Therefore, the [determine_A_stat_emg()](modules.rst#emgfit.spectrum.spectrum.determine_A_stat_emg) method can be used to estimate $A_{stat,emg}$ for the specific peak shape in the spectrum. This is done by fitting many simulated spectra created via bootstrap re-sampling from a reference peak in the spectrum. The reference peak should be well-separated and have decent statistics (e.g. the peak-shape calibrant). For details on how $A_{stat,emg}$ is estimated see the linked docs of [determine_A_stat_emg()](modules.rst#emgfit.spectrum.spectrum.determine_A_stat_emg).

This method will typically run for ~10 minutes if `N_spectra=1000` (default) is used. For demonstration purposes here the number of bootstrapped spectra generated for each data point (`N_spectra` argument) was reduced to 10 to get a quicker run time. This is also the reason for the large scatter of the data points below. 

In practice it is convenient to skip this method for the first processing of the spectrum since this will only affect the statistical uncertainties but no other fit properties. Once reasonable fits have been achieved for all peaks of interest in the cells below, the exact uncertainties can be obtained by returning to this cell to execute [determine_A_stat_emg()](modules.rst#emgfit.spectrum.spectrum.determine_A_stat_emg) with a decent value for `N_spectra`  and then re-runnning the cells below (then with the update value for [determine_A_stat_emg()](modules.rst#emgfit.spectrum.spectrum.determine_A_stat_emg)). The latter is conveniently done by using the `Run All Below` option in the `Cell` panel of the Jupyter Notebook. 

In [ ]:
# Determine A_stat_emg and save the resulting plot
# In actual practice N_spectra >= 1000 should be used
spec.determine_A_stat_emg(species='Ca43:F19:-1e', x_range=0.004, plot_filename='outputs/'+filename+'_MLE', N_spectra=10) 

## Fit all peaks, perform mass re-calibration & obtain final mass values
The following code fits all peaks in the spectrum, performs the mass (re-)calibration, determines the peak-shape uncertainties and updates the peak properties list with the results including the final mass values and their uncertainties. 

The simultaneous mass recalibration is optional and only invoked when the `species_mass_calib` (or the `index_mass_calib`) argument are specified. If this feature is not used, the [fit_peaks()](modules.rst#emgfit.spectrum.spectrum.fit_peaks) method requires a pre-existing mass calibration (see Alternative 1 section below). In contrast to [determine_peak_shape()](modules.rst#emgfit.spectrum.spectrum.determine_peak_shape), by default [fit_peaks()](modules.rst#emgfit.spectrum.spectrum.fit_peaks) performs a binned maximum likelihood fit ('MLE'). For chi-square fitting with [fit_peaks()](modules.rst#emgfit.spectrum.spectrum.fit_peaks) see Alternative 2 section below. Fits with [fit_peaks()](modules.rst#emgfit.spectrum.spectrum.fit_peaks) can be restricted to a user-defined mass range or to groups of neighbouring peaks selected by index (see the commented-out lines below). 

In [ ]:
# Maximum likelihood fit of all peaks in the spectrum
spec.fit_peaks(species_mass_calib='Ti46:O16:-1e') 

# Maximum likelihood fit of peaks in a user-defined mass range
#spec.fit_peaks(species_mass_calib='Ti46:O16:-1e',x_fit_cen=61.9455,x_fit_range=0.01)

# Maximum likelihood fit of peaks specified by index
#spec.fit_peaks(species_mass_calib='Ti46:O16:-1e',peak_indeces=[3,4,5])

## Plot the fit curve zoomed to a region of interest (optional)
For more detailed inspection of the fit, a zoom to peaks or regions of interest can be shown with the [plot_fit_zoom()](modules.rst#emgfit.spectrum.spectrum.plot_fit_zoom) method.  

In [ ]:
spec.plot_fit_zoom(peak_indeces=[3,4]) # zoom to region around peaks 3 and 4 

## Perform parametric bootstrap to get refined statistical uncertainties (optional)
The `A_stat_emg` determination with [determine_A_stat_emg()](modules.rst#emgfit.spectrum.spectrum.determine_A_stat_emg) relies on fits of bootstrapped subspectra of a single reference peak. The obtained A_stat_emg factor is then used to estimate the statistical uncertainties of all peaks. 

As an alternative, the statistical uncertainty can be estimated for each peak individually using the [get_errors_from_resampling()](modules.rst#emgfit.spectrum.spectrum.get_errors_from_resampling) method. In this method synthetic spectra are created for all peaks of interest by resampling events from the best-fit curve ("parametric bootstrap"). As opposed to the non-parametric bootstrap of [determine_A_stat_emg()](modules.rst#emgfit.spectrum.spectrum.determine_A_stat_emg), this technique is also applicable to low statistics peaks (assuming that the fit model describes the data well). The fits of the peaks of interest are re-performed using a large number of synthetic spectra (by default: `N_spectra`=1000) and the statistical mass and area uncertainties are estimated using the standard deviations of the obtained fit results. Finally, the original statistical mass and area uncertainties in the peak properties table are overwritten with the new values. 

In [ ]:
# NOTE: For quicker run time in this demo, the number of synthetic spectra to fit `N_spectra` was manually reduced. 
# For reliable results run this method with at least the default value of N_spectra=1000

spec.get_errors_from_resampling(N_spectra=20) # arguments adapted for demonstration 
#spec.get_errors_from_resampling() # typical execution with default arguments

## Get refined peak-shape uncertainties using MCMC parameter samples (optional)
The default peak-shape error estimation in [fit_peaks()](modules.rst#emgfit.spectrum.spectrum.fit_peaks) relies on some simplifying assumptions:

1. The posterior distributions of the shape parameters follow normal distributions. 
2. The shape parameters are uncorrelated. 

In many cases, at least one of those assumptions is violated. Therefore, a refined way of estimating the peak-shape uncertainties has been added to emgfit: [get_MC_peakshape_errors()](modules.rst#emgfit.spectrum.spectrum.get_MC_peakshape_errors). This method uses Markov-Chain Monte Carlo (MCMC) sampling to estimate the posterior distributions of the shape parameters. The sampling results are compiled in a corner plot/"covariance map" which includes both 1D-histograms of the parameter posteriors and 2D-histograms of the parameter correlations. By randomly drawing shape parameters sets from the obtained MCMC samples one obtains a representation of all peak shapes supported by the data. The calibrant and the peaks of interest are then re-fitted with all drawn shape parameter sets. Refined peak-shape uncertainties are obtained from the RMS deviation of the resulting mass values and peak areas from the best-fit values obtained with [fit_peaks()](modules.rst#emgfit.spectrum.spectrum.fit_peaks). Usually, accounting for parameter correlations results in significantly smaller peak-shape errors.

The MCMC sampling can also already be performed during the peak-shape calibration using the `map_par_covar` option of 
[determine_peak_shape()](modules.rst#emgfit.spectrum.spectrum.determine_peak_shape). The corner plot of the parameter covariances can be used to assess whether [get_MC_peakshape_errors()](modules.rst#emgfit.spectrum.spectrum.get_MC_peakshape_errors) should be run. For more details on the MC peak-shape uncertainty estimation see docs of [get_MC_peakshape_errors()](modules.rst#emgfit.spectrum.spectrum.get_MC_peakshape_errors).

In [ ]:
# NOTE: For quicker run time in this demo, the length of the sampling chain, the thinning interval and the number 
# of shape parameter sets to perform fits with were manually reduced with the `steps`, `thin` and 
# `N_samples` arguments, respectively. This triggers a warning about the insufficient MCMC chain length. 
# For reasonable results those parameters should be increased to at least their default values. 
# For this specific data decent results are obtained using the following: steps = 16000, thin = 280, N_samples = 1000

spec.get_MC_peakshape_errors(steps=1000, thin=20, N_samples=50) # arguments adapted for demonstration
#spec.get_MC_peakshape_errors() # typical execution with default arguments

## Export fit results
Once all peaks have been fitted, the results can be exported to an XLSX Excel file and a separate file with the peak-shape calibration parameters by using the [save_results()](modules.rst#emgfit.spectrum.spectrum.save_results) method. The XLSX file contains three sheets: 

1. General properties of the spectrum object, e.g. input filename, used versions of emgfit and all relevant dependencies  
2. The peak properties table with all fit results along with linear and logarithmic plots of all peak fits
3. The mass shifts obtained via $\pm1\sigma$ variation of the shape parameters in the default peak-shape uncertainty evaluation 

In [ ]:
spec.save_results('outputs/'+filename+' fitting MLE')

**That's it! In principle we're be done with the fitting at this point.**  Next we would probably take a look at the output file and proceed with some post-processing in Excel (e.g. combining mass values from different spectra etc.). 

However, since emgfit gives the user a large amount of freedom, there's are a number of things that could have been done differently depending on your preferences. So here is some possible...  





## Alternative procedures:

The above steps represent a full spectrum analysis. However, emgfit gives the user the freedom to take many different routes in processing the spectrum. Some of the possible alternatives are presented in the following: 

### Alternative 1: Performing the mass recalibration separately before the ion-of-interest fits
All steps up to the final peak fit are identical. For breviety here we simply create an exact clone of the above spectrum object:

In [ ]:
import copy
spec2 = copy.deepcopy(spec) # create a clone of the spectrum object

#### First obtain the recalibration factor from a fit of the mass calibrant

In [ ]:
spec2.fit_calibrant(species_mass_calib='Ti46:O16:-1e', show_fit_report=False)

#### Fit all peaks and use the mass recalibration factor from above to calculate the final mass values

In [ ]:
spec2.fit_peaks(show_fit_report=False)

### Alternative 2: Chi-square instead of MLE fitting 
All steps up to the final peak fit are identical. For breviety here we simply create an exact clone of the above spectrum object and re-use the above peak-shape calibration (obtained with chi-square fitting):

In [ ]:
import copy
spec_chi_sq = copy.deepcopy(spec) # create a clone of the spectrum object

# Use Pearson's chi-squared statistic for A_stat_emg determination
spec_chi_sq.determine_A_stat_emg(species='Ca43:F19:-1e', x_range=0.004, cost_func='chi-square', 
                                 plot_filename='outputs/'+filename+'_chi-square', N_spectra=10) 

In [ ]:
# Fit with Pearson's chi-squared statistic as cost function
spec_chi_sq.fit_peaks(species_mass_calib='Ti46:O16:-1e', cost_func='chi-square', 
                      show_fit_report=False) 